In [ ]:
import pandas as pd
cal = pd.read_csv('data/raw/calendar2024.csv')
lis = pd.read_csv('data/raw/listings2024.csv') 
rev = pd.read_csv('data/raw/reviews2024.csv')
print("Listings shape:", lis.shape)
print("Calendar shape:", cal.shape) 
print("Reviews shape:", rev.shape)

## Cleaning

In [ ]:
null_cols_lis = lis.columns[lis.isna().all()].tolist()
lis = lis.drop(columns=null_cols_lis)

lis.drop(columns=['scrape_id', 'host_name', 'picture_url', 'host_url', 'host_thumbnail_url', 'host_picture_url'], inplace=True)
cal.drop(columns=['adjusted_price'], inplace=True)
rev.drop(columns=['reviewer_name'], inplace=True)


def convert_to_boolean(df, columns, true_value='t'):
    """Convert specified columns from string indicators to boolean"""
    for col in columns:
        df[col] = df[col] == true_value
    return df

boolean_cols = ['instant_bookable', 'host_is_superhost', 'host_has_profile_pic', 'host_identity_verified', 'has_availability']
lis = convert_to_boolean(lis, boolean_cols)
cal['available'] = cal['available'] == 't'


def convert_to_datetime(df, columns):
    """Convert specified columns to datetime"""
    for col in columns:
        df[col] = pd.to_datetime(df[col])
    return df

datetime_cols_lis = ['calendar_last_scraped', 'first_review', 'last_review', 'last_scraped', 'host_since']
lis = convert_to_datetime(lis, datetime_cols_lis)
cal['date'] = pd.to_datetime(cal['date'])
rev['date'] = pd.to_datetime(rev['date'])


def convert_to_type(df, columns, dtype):
    """Convert specified columns to given dtype"""
    for col in columns:
        df[col] = df[col].astype(dtype)
    return df

string_columns = ['bathrooms_text', 'neighbourhood', 'neighbourhood_cleansed', 'property_type', 'room_type', 'host_location', 'host_about', 'host_neighbourhood', 'listing_url', 'host_response_time', 'source', 'name','description','neighborhood_overview']
lis = convert_to_type(lis, string_columns, "string")
rev['comments'] = rev['comments'].astype("string")


percentage_cols = ['host_response_rate', 'host_acceptance_rate']
for col in percentage_cols:
    lis = lis.rename(columns={col: f"{col}_pct"})
    lis[f"{col}_pct"] = lis[f"{col}_pct"].str.rstrip('%').astype('float') / 100

# Currency inconsistency adjustment 
lis['price'] = lis['price'].str.replace(r'[\$,]', '', regex=True)
lis = lis.rename(columns={'price': 'price_DKK'})
lis['price_DKK'] = pd.to_numeric(lis['price_DKK'], errors='coerce')

cal['price'] = cal['price'].str.replace(r'[\$,]', '', regex=True)
cal = cal.rename(columns={'price': 'price_USD'})
cal['price_USD'] = pd.to_numeric(cal['price_USD'], errors='coerce')

########## Handling list columns ##########

# Count amenities and add new column
lis['amenities_count'] = lis.amenities.str.strip('[]').str.split(',').str.len()

# Encoding host_verifications
# First clean up the string representation of lists
lis['host_verifications'] = lis['host_verifications'].str.strip('[]').str.replace("'", "").str.split(', ')
# Create one-hot encoded columns
verification_dummies = lis['host_verifications'].str.join('|').str.get_dummies()
# Add prefix to avoid column name conflicts
verification_dummies = verification_dummies.add_prefix('verification_')
# Join with original dataframe if needed
lis = pd.concat([lis, verification_dummies], axis=1)

print("Listings shape:", lis.shape)
print("Calendar shape:", cal.shape) 
print("Reviews shape:", rev.shape)

In [6]:
# Save processed datasets
lis.to_csv('data/processed/02_listings_cleaned.csv', index=False)
cal.to_csv('data/processed/02_calendar_cleaned.csv', index=False) 
rev.to_csv('data/processed/02_reviews_cleaned.csv', index=False)
